# Vehicle detection using YOLO 8 and YOLO 11 on the kitti dataset

> ❗Do not forget to set the Runtime to **T4 GPU** when using Google Colab

> ❗This notebook will download more than **23 Gb** of data

> ❗This notebook will use more than **30 Gb** of storage

> ❗This notebook will take more than **80 hours** to run on an Nvidia RTX 4070 CUDA enabled GPU



## Setup

### Imports

In [ ]:
!pip install ultralytics -q

In [ ]:
from ultralytics import YOLO
import numpy as np
import os
import matplotlib.pyplot as plt
from pathlib import Path
import json
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import pandas as pd
from PIL import Image
import kagglehub
import shutil

### Consts

In [ ]:
# Set some useful const
DATA_DIR = Path('data').resolve()
KAGGLE_DIR = DATA_DIR / 'kaggle'
WORKING_DIR = DATA_DIR / 'working'
OUTPUT_DIR = DATA_DIR / 'output'
RESULTS_DIR = DATA_DIR / 'results'

kitti_path = KAGGLE_DIR / 'kitti-dataset'
kitti_yolo_path = KAGGLE_DIR / 'kitti-dataset-yolo-format'

**Define labels that should be kept and their mappings**

```PYTHON
KEEP_LABELS = {
  '[Current class id | str]': ('[Class name | str]', 'New class id (starting from 0) | str'),
  }
```

Ex for only motor vehicles:
```PYTHON
KEEP_LABELS = {
  '0': ('Car', '0'),
  '2': ('Van', '1'),
  '4': ('Truck', '2'),
  }
```

In [ ]:
KEEP_LABELS = {
  '0': ('Car', '0'),
  '2': ('Van', '1'),
  '4': ('Truck', '2'),
  }

In [ ]:
# Disable WANDB integration
%env WANDB_DISABLED=True

### Download datasets from kaggle

**!! WARNING !! more than 22 Gb**

In [ ]:
# Download kitti dataset
path = kagglehub.dataset_download('klemenko/kitti-dataset')
print("Path to downloaded dataset:", path)

# Move files to kaggle directory
kitti_path = shutil.move(path, KAGGLE_DIR / 'kitti-dataset')
kitti_path = Path(kitti_path)
print("Dataset moved to:", kitti_path)

In [ ]:
# Download kitti-yolo-labels dataset
path = kagglehub.dataset_download('shreydan/kitti-dataset-yolo-format')
print("Path to downloaded dataset:", path)

# Move files to kaggle directory
kitti_yolo_path = shutil.move(path, KAGGLE_DIR / 'kitti-dataset-yolo-format')
kitti_path = Path(kitti_yolo_path)
print("Dataset moved to:", kitti_yolo_path)

### Preparing Dataset

In [ ]:
# Define image and label paths
img_path = kitti_path / 'data_object_image_2' / 'training' / 'image_2'
label_path = kitti_yolo_path / 'labels'

In [ ]:
# Get class definition
with open(kitti_yolo_path / 'classes.json','r') as f:
    classes = json.load(f)

# Display classes
classes

In [ ]:
# Get all image and label paths and pair them together
ims = sorted(list(img_path.glob('*')))
labels = sorted(list(label_path.glob('*')))
pairs = list(zip(ims,labels))

## Display to examples
pairs[:2]

#### Preparing File Structure

```
/working
    |
    -train
    |   |
    |   -000000.png
    |   -000000.txt
    |   ...
    |
    -val
      |
      -000001.png
      -000001.txt
      ...
```

##### Split dataset

In [ ]:
# Split datatest into train and test
train_dataset, test_dataset = train_test_split(pairs, test_size=0.1, shuffle=True)
len(train_dataset), len(test_dataset)

##### Create directories

In [ ]:
# create working directory and train / validation directories
WORKING_DIR.mkdir(exist_ok=True)

train_path = WORKING_DIR / 'train'
train_path.mkdir(exist_ok=True)

valid_path = WORKING_DIR / 'valid'
valid_path.mkdir(exist_ok=True)

##### Copy files to directories

In [ ]:
# Copy labels and images from train subset to train directory
for t_img, t_lb in tqdm(train_dataset):
    im_path = train_path / t_img.name
    lb_path = train_path / t_lb.name
    shutil.copy(t_img,im_path)
    shutil.copy(t_lb,lb_path)

In [ ]:
# copy labels and images from test subset to validation directory
for t_img, t_lb in tqdm(test_dataset):
    im_path = valid_path / t_img.name
    lb_path = valid_path / t_lb.name
    shutil.copy(t_img,im_path)
    shutil.copy(t_lb,lb_path)

##### Clean up dataset from unwanted labels

In [ ]:
# Define a function cleanup the dataset from unwanted labels
def filter_labels_from_label_files(directory, keep_labels):
    """Opens every .txt file in a directory and removes lines not starting with keys from keep_labels."""
    for filename in os.listdir(directory):
        if filename.endswith('.txt'):
            filepath = os.path.join(directory, filename)
            with open(filepath, 'r') as f:
                lines = f.readlines()
            with open(filepath, 'w') as f:
                for line in lines:
                    parts = line.strip().split()
                    if parts[0] in keep_labels:
                        # Access second element of the tuple in the keep_labels dictionary where new label index is saved
                        parts[0] = keep_labels[parts[0]][1]
                        corrected_line = ' '.join(parts) + '\n'
                        f.write(corrected_line)

In [ ]:
# Cleanup the train dataset from unwanted labels
filter_labels_from_label_files(train_path, KEEP_LABELS)

In [ ]:
# Cleanup the test dataset from unwanted labels
filter_labels_from_label_files(valid_path, KEEP_LABELS)

#### Create YOLO YAML dataset description file

In [ ]:
# create yaml file for yolo
yaml_file = 'names:\n'
yaml_file += '\n'.join(f'  {value[1]}: {value[0]}' for key, value in KEEP_LABELS.items())
yaml_file += f'\ntrain: {str(train_path)}\nval: {str(valid_path)}'
with open(WORKING_DIR / 'kitti.yaml','w') as f:
    f.write(yaml_file)

## Train Models

!! WARNING !! model training takes around 20 hours per model on a high performance GPU (RTX 4070)

Results are saved in the output directory under the respective model name ex: `data/output/yolo_8_n`

Detection COCO 80 pre-trained classes

| **Model**                                                                            | **Size (pixels)** | **mAPval 50-95** | **Speed CPU ONNX (ms)** | **params (M)** | **FLOPs (B)** |
|--------------------------------------------------------------------------------------|-------------------|------------------|-------------------------|----------------|---------------|
| [YOLOv8n](https://github.com/ultralytics/assets/releases/download/v8.2.0/yolov8n.pt) | 640               | 37.3             | 80.4                    | 3.2            | 8.7           |
| [YOLOv8x](https://github.com/ultralytics/assets/releases/download/v8.2.0/yolov8x.pt) | 640               | 53.9             | 479.1                   | 68.2           | 257.8         |
| [YOLO11n](https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo11n.pt) | 640               | 39.5             | 56.1 ± 0.8              | 2.6            | 6.5           |
| [YOLO11x](https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo11x.pt) | 640               | 54.7             | 462.8 ± 6.7             | 56.9           | 194.9         |



#### Setup

In [ ]:
# Common arguments
DEVICE = 0
DATA = WORKING_DIR / 'kitti.yaml'
MIXUP = 0.2
MOSAIC = 0.2
PRETRAINED = True

RANDOM_PREDICTION_LIST = [test_dataset[idx][0] for idx in np.random.randint(0,len(test_dataset),(10,))]

#### YOLO v8 - N

##### Model definition

In [ ]:
# Loading pretrained model yolo 8 N
# Model is downloaded if it does not exist already
model_yolo8_n = YOLO(WORKING_DIR / 'yolov8n.pt')

##### Training

In [ ]:
yolo8_n_train_results = model_yolo8_n.train(
    project=OUTPUT_DIR / 'yolo_8_n',
    data=DATA,
    epochs=300,
    patience=15,
    mixup=MIXUP,
    mosaic=MOSAIC,
    device=DEVICE,
    pretrained=PRETRAINED,
)

##### Validation

In [ ]:
yolo8_n_valid_results = model_yolo8_n.val()

##### Sample predictions

In [ ]:
yolo8_n_predictions = model_yolo8_n.predict(RANDOM_PREDICTION_LIST,save=True)

#### YOLO v8 - X

##### Model definition

In [ ]:
# Loading pretrained model yolo 8 X
# Model is downloaded if it does not exist already
model_yolo8_x = YOLO(WORKING_DIR / 'yolov8x.pt')

##### Training

In [ ]:
yolo8_x_train_results = model_yolo8_x.train(
    project=OUTPUT_DIR / 'yolo_8_x',
    data=DATA,
    epochs=50,
    patience=3,
    mixup=MIXUP,
    mosaic=MOSAIC,
    device=DEVICE,
    pretrained=PRETRAINED
)

##### Validation

In [ ]:
yolo8_x_valid_results = model_yolo8_x.val()

##### Sample predictions

In [ ]:
yolo8_x_predictions = model_yolo8_x.predict(RANDOM_PREDICTION_LIST,save=True)

#### YOLO 11 - N

##### Model definition

In [ ]:
# Loading pretrained model yolo 11 X
# Model is downloaded if it does not exist already
model_yolo11_n = YOLO(WORKING_DIR / 'yolo11n.pt')

##### Training

In [ ]:
yolo11_n_train_results = model_yolo11_n.train(
    project=OUTPUT_DIR / 'yolo_11_n',
    data=DATA,
    epochs=300,
    patience=15,
    mixup=MIXUP,
    mosaic=MOSAIC,
    device=DEVICE,
    pretrained=PRETRAINED,
)

##### Validation

In [ ]:
yolo11_n_valid_results = model_yolo11_n.val()

##### Sample predictions

In [ ]:
yolo11_n_predictions = model_yolo11_n.predict(RANDOM_PREDICTION_LIST,save=True)

#### YOLO 11 - X

##### Model definition

In [ ]:
# Loading pretrained model yolo 11 X
# Model is downloaded if it does not exist already
model_yolo11_x = YOLO(WORKING_DIR + '/yolo11x.pt')

##### Training

In [ ]:
yolo11_x_train_results = model_yolo11_x.train(
    project=OUTPUT_DIR / 'yolo_11_x',
    data=DATA,
    epochs=50,
    patience=3,
    mixup=MIXUP,
    mosaic=MOSAIC,
    device=DEVICE,
    pretrained=PRETRAINED
)

##### Validation

In [ ]:
yolo11_x_valid_results = model_yolo11_x.val()

##### Sample predictions

In [ ]:
yolo11_x_predictions = model_yolo11_x.predict(RANDOM_PREDICTION_LIST,save=True)

## Display results

### Get results directories

In [ ]:
models_output = os.listdir(OUTPUT_DIR)
for model_id, model in enumerate(models_output):
    print(id, model)

### Select model

In [ ]:
# select model
selected_model = models_output[0]

selected_output_path_1 = OUTPUT_DIR / selected_model / 'train'
selected_output_path_2 = OUTPUT_DIR / selected_model / 'train2'
selected_output_path_3 = OUTPUT_DIR / selected_model / 'train3'

### Stats

#### General stats

In [ ]:
plt.figure(figsize=(10,20))
plt.imshow(Image.open(selected_output_path_1/ 'results.png'))
plt.axis('off')
plt.show()

#### Confusion Matrix

##### Absolut

In [ ]:
plt.figure(figsize=(10,20))
plt.imshow(Image.open(selected_output_path_1 / 'confusion_matrix.png'))
plt.axis('off')
plt.show()

##### Normalized

In [ ]:
plt.figure(figsize=(10,20))
plt.imshow(Image.open(selected_output_path_1 / 'confusion_matrix_normalized.png'))
plt.axis('off')
plt.show()

#### Other Stats

##### F1-Confidence Curve

In [ ]:
plt.figure(figsize=(10,20))
plt.imshow(Image.open(selected_output_path_1 / 'F1_curve.png'))
plt.axis('off')
plt.show()

##### Precision-Confidence Curve

In [ ]:
plt.figure(figsize=(10,20))
plt.imshow(Image.open(selected_output_path_1 / 'P_curve.png'))
plt.axis('off')
plt.show()

##### Recall-Confidence Curve

In [ ]:
plt.figure(figsize=(10,20))
plt.imshow(Image.open(selected_output_path_1 / 'R_curve.png'))
plt.axis('off')
plt.show()

##### Precision-Recall Curve

In [ ]:
plt.figure(figsize=(10,20))
plt.imshow(Image.open(selected_output_path_1 / 'PR_curve.png'))
plt.axis('off')
plt.show()

#### Stats per epochs

In [ ]:
df_results = pd.read_csv(selected_output_path_1 / 'results.csv', sep=",")
df_results

### Results

#### Validation

Images on the left are the labeled image from the dataset the right images are the predictions

In [ ]:
fig, axes = plt.subplots(ncols=2, nrows=2)
images = [
    selected_output_path_2 / 'val_batch0_labels.jpg',
    selected_output_path_2 / 'val_batch0_pred.jpg',
    selected_output_path_2 / 'val_batch1_labels.jpg',
    selected_output_path_2 / 'val_batch1_pred.jpg']

for ax, img in zip(axes.flatten(), images):
    ax.imshow(Image.open(img))
    ax.axis('off')

fig.set_size_inches(40,15)
plt.subplots_adjust(wspace=0.02, hspace=0)
plt.show()

#### Random predictions

In [ ]:
# Get predicction images
predictions = list(selected_output_path_3.glob('*'))
predictions

In [ ]:
# Display prediction images
def plot_images(prediction_images):
    num_images = len(prediction_images)
    rows = num_images
    cols = 1
    fig, axes = plt.subplots(rows, cols, figsize=(15, 40))
    for ax in axes.flat:
        ax.axis('off')
    for i, img_path in enumerate(prediction_images):
        img = Image.open(img_path)
        axes[i].imshow(img)

    plt.tight_layout()
    plt.show()

plot_images(predictions)

## Save and move results

### Get current date

In [ ]:
from datetime import datetime
now = datetime.now()

# Format the date and time as a string
date_time_str = now.strftime('%Y-%m-%d_%H-%M-%S')
print(date_time_str)

### Create directory

In [ ]:
# Create a directory named after the current date and time
current_result_path = RESULTS_DIR / date_time_str
valid_path = current_result_path.resolve()
valid_path.mkdir(exist_ok=True)

### Get folders in output dir

In [ ]:
models_output = os.listdir(OUTPUT_DIR)
models_output

### Copy output

In [ ]:
# Copy output folders to results
for output_folder_name in models_output:
    current_output_path = OUTPUT_DIR / output_folder_name
    if os.path.exists(current_output_path):
        shutil.move(current_output_path, current_result_path / output_folder_name)

## Cleanup

### General

In [ ]:
shutil.rmtree(WORKING_DIR)

In [ ]:
shutil.rmtree(OUTPUT_DIR)

### Further

In [ ]:
shutil.rmtree(KAGGLE_DIR)

 # Appendix

## Convert YOLO Dataset to COCO for other models

### Setup

In [ ]:
!pip install globox

In [ ]:
import globox
from pathlib import Path

In [ ]:
COCO_ANNOTATIONS_PATH = WORKING_DIR / 'annotation'

In [ ]:
COCO_ANNOTATIONS_PATH.mkdir(exist_ok=True)

### Create .name file

In [ ]:
yolo_name_path = WORKING_DIR / 'yolo.name'

In [ ]:
with open(yolo_name_path, 'w') as name_file:
    for index, label in KEEP_LABELS.items():
        print(label[0])
        name_file.write(label[0]+ "\n")

In [ ]:
# Load .name file with globox
yolo_names = globox.AnnotationSet.parse_names_file(yolo_name_path)

### Convert train dataset

In [ ]:
# Load dataset into globox
yolo_train = globox.AnnotationSet.from_yolo_v5(
    folder=train_path,
    image_folder=train_path)

In [ ]:
# Add labels into globox
yolo_train.map_labels(yolo_names)

In [ ]:
# Show information about the dataset
yolo_train.show_stats()

In [ ]:
# Convert the dataset to the coco format
yolo_train.save_coco(
    COCO_ANNOTATIONS_PATH / 'coco_train.json',
    auto_ids=True
)

### Convert test dataset

In [ ]:
# Load dataset into globox
yolo_test = globox.AnnotationSet.from_yolo_v5(
    folder=valid_path,
    image_folder=valid_path)

In [ ]:
# Add labels into globox
yolo_test.map_labels(yolo_names)

In [ ]:
# Show information about the dataset
yolo_test.show_stats()

In [ ]:
# Convert the dataset to the coco format
yolo_test.save_coco(
    COCO_ANNOTATIONS_PATH / 'coco_valid.json',
    auto_ids=True
)